In [160]:
%matplotlib inline

import torch
from PIL import Image

IS_GPU = torch.cuda.is_available()

In [161]:
# Show sample images

In [162]:
class DDSM(torch.utils.data.Dataset):
    def __init__(self, root, label_file, transform):
        self.root = root
        self.transform = transform
        self.image_list = []
        
        def process_line(line):
            img_name, label = line.strip().split(' ')
            label = int(label)
            return img_name, label
        
        with open(os.path.join(root, label_file), 'r') as f:
            for line in f.readlines():
                self.image_list.append(process_line(line))
            
    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self, idx):
        img_name, label = self.image_list[idx]
        img = Image.open(os.path.join(self.root, img_name)).convert('RGB')
        img = self.transform(img)
        return img, label

In [129]:
# Calculate mean and std in advance

import numpy as np
import torchvision.transforms as transforms

train_dataset = DDSM(root, 'images/dataset-v2/train.txt', transforms.Compose([transforms.Resize(224),
                                                                              transforms.ToTensor()],))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=False, num_workers=4, pin_memory=True)

data_mean = []
data_std = []

for batch in train_loader:
    np_images = batch[0].numpy()
    batch_mean = np.mean(np_images, axis=(0, 2, 3))
    batch_std = np.std(np_images, axis=(0, 2, 3))
    data_mean.append(batch_mean)
    data_std.append(batch_std)

data_mean = np.array(data_mean).mean(axis=0)
data_std = np.array(data_std).mean(axis=0)

print('mean: {}'.format(data_mean))
print('std: {}'.format(data_std))


mean: [0.5039065 0.5039065 0.5039065]
std: [0.1724488 0.1724488 0.1724488]


In [163]:
# Dataloader

import os
import torchvision.transforms as transforms

root = './'

normalize = transforms.Normalize(mean=[0.504, 0.504, 0.504], std=[0.172, 0.172, 0.172])

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    normalize,
])

augmented_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(360),
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0)),
    transforms.ToTensor(),
    normalize,
])

train_dataset = DDSM(root, 'images/dataset-v2/train.txt', augmented_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

test_dataset = DDSM(root, 'images/dataset-v2/test.txt', transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

In [164]:
print(len(train_loader))
print(len(test_loader))

119
30


In [208]:
# Baseline CNN

import torch.nn as nn
import torch.nn.functional as F

class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()
        self.conv_net = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(2, 2),
        )
        
        self.fc_net = nn.Sequential(
            nn.Linear(in_features=288, out_features=512),
            nn.Linear(in_features=512, out_features=128),
            nn.Linear(in_features=128, out_features=2),
        )
        
    def forward(self, x):
        x = self.conv_net(x)
        x = x.view(x.size(0), -1) # batch_size x image
        x = self.fc_net(x)
        
        return x


In [209]:
# Baseline CNN

def get_basenet():
    model = BaseNet()

    if IS_GPU:
        model = model.cuda()

    model.eval()
    
    return model

In [210]:
# AlexNet

def get_alexnet(pretrained=False):
    import torchvision.models as models
    import torch.nn as nn

    model = models.__dict__['alexnet'](pretrained=False)

    model.classifier._modules['6'] = nn.Linear(4096, 2)

    if IS_GPU:
        model.features = torch.nn.DataParallel(model.features)
        model.cuda()

    model.eval()
    
    return model

In [211]:
# ResNet-18

def get_resnet(pretrained=False):
    import torchvision.models as models
    import torch.nn as nn

    model = models.__dict__['resnet18'](pretrained=pretrained)

    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 2)

    if IS_GPU:
        model = torch.nn.DataParallel(model)
        model.cuda()

    model.eval()
    
    return model

In [212]:
# Define optimizer

def get_optimizer(model, lr=0.01, momentum=0.9):
    
    import torch.optim as optim

    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    
    return optimizer

In [213]:
# Train Model

def train_model(model, optimizer, start_epoch=1, epoch_num=20, filename=None, save_model_name=None):
    import time

    # filename = 'resnet-no-tl.txt'

    start_ts = time.time()

    best_test_acc = 0.0
    best_epoch = 0

    for epoch in range(start_epoch, start_epoch + epoch_num):
        epoch_loss = 0.0
        epoch_accuracy = 0.0

        sample_count = 0
        correct_count = 0

        for i, samples in enumerate(train_loader):
            images, labels = samples
            if IS_GPU:
                images = images.cuda()
                labels = labels.cuda()

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs.data, 1)

            epoch_loss += loss.item()

            sample_count += len(labels)
            correct_count += torch.sum(preds == labels).item()

        print('Time: {}'.format(int(time.time() - start_ts)))

        train_acc = correct_count / sample_count
        test_acc, precision, recall, F1 = evaluate(model, test_loader, epoch)

        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_epoch = epoch

        print('Epoch {} '.format(epoch) + 'tarin accuracy: ' + str(train_acc))
        print('Epoch {} '.format(epoch) + 'loss: ' + str(epoch_loss))
        print('Epoch {} '.format(best_epoch) + 'has the best test accuracy: ' + str(best_test_acc))
        print()

        if filename:
            with open('./evaluation/{}'.format(filename), 'a') as f:
                f.write('Evaluate epoch {} '.format(epoch) + 'accuracy: ' + str(test_acc) + '\n')
                f.write('Evaluate epoch {} '.format(epoch) + 'precision: ' + str(precision) + '\n')
                f.write('Evaluate epoch {} '.format(epoch) + 'recall: ' + str(recall) + '\n')
                f.write('Evaluate epoch {} '.format(epoch) + 'F1: ' + str(F1) + '\n')
                f.write('epoch {} '.format(epoch) + 'tarin accuracy: ' + str(train_acc) + '\n')
                f.write('epoch {} '.format(epoch) + 'loss: ' + str(epoch_loss) + '\n')
                f.write('epoch {} '.format(best_epoch) + 'has the best test acc: ' + str(best_test_acc) + '\n\n')
                
    if save_model_name:
        torch.save(model, './models/{}.pth'.format(save_model_name))
        

In [202]:
# model = get_resnet()
# optimizer = get_optimizer(model)
# train_model(model, optimizer, start_epoch=451, epoch_num=100, filename='resnet-no-tl.txt')
train_model(model, optimizer, start_epoch=451, epoch_num=100)

Time: 14
Evaluate epoch 451 accuracy: 0.8328042328042328
Evaluate epoch 451 precision: 0.8165548098434005
Evaluate epoch 451 recall: 0.8276643990929705
Evaluate epoch 451 F1: 0.822072072072072
Epoch 451 tarin accuracy: 0.9574074074074074
Epoch 451 loss: 14.375742992386222
Epoch 451 has the best test accuracy: 0.8328042328042328

Time: 31
Evaluate epoch 452 accuracy: 0.8423280423280424
Evaluate epoch 452 precision: 0.8244444444444444
Evaluate epoch 452 recall: 0.8412698412698413
Evaluate epoch 452 F1: 0.8327721661054994
Epoch 452 tarin accuracy: 0.9529100529100529
Epoch 452 loss: 14.14082377590239
Epoch 452 has the best test accuracy: 0.8423280423280424

Time: 49
Evaluate epoch 453 accuracy: 0.8317460317460318
Evaluate epoch 453 precision: 0.8092105263157895
Evaluate epoch 453 recall: 0.8367346938775511
Evaluate epoch 453 F1: 0.8227424749163879
Epoch 453 tarin accuracy: 0.9542328042328042
Epoch 453 loss: 15.204109400510788
Epoch 452 has the best test accuracy: 0.8423280423280424

Time: 

Time: 450
Evaluate epoch 476 accuracy: 0.8793650793650793
Evaluate epoch 476 precision: 0.8724373576309795
Evaluate epoch 476 recall: 0.8684807256235828
Evaluate epoch 476 F1: 0.8704545454545455
Epoch 476 tarin accuracy: 0.9603174603174603
Epoch 476 loss: 12.327671254053712
Epoch 476 has the best test accuracy: 0.8793650793650793

Time: 468
Evaluate epoch 477 accuracy: 0.8486772486772487
Evaluate epoch 477 precision: 0.8267543859649122
Evaluate epoch 477 recall: 0.854875283446712
Evaluate epoch 477 F1: 0.8405797101449274
Epoch 477 tarin accuracy: 0.958994708994709
Epoch 477 loss: 13.627521509304643
Epoch 476 has the best test accuracy: 0.8793650793650793

Time: 485
Evaluate epoch 478 accuracy: 0.8423280423280424
Evaluate epoch 478 precision: 0.827354260089686
Evaluate epoch 478 recall: 0.8367346938775511
Evaluate epoch 478 F1: 0.8320180383314543
Epoch 478 tarin accuracy: 0.9547619047619048
Epoch 478 loss: 12.959946038201451
Epoch 476 has the best test accuracy: 0.8793650793650793

Time

Time: 892
Evaluate epoch 501 accuracy: 0.8433862433862434
Evaluate epoch 501 precision: 0.8262806236080178
Evaluate epoch 501 recall: 0.8412698412698413
Evaluate epoch 501 F1: 0.8337078651685392
Epoch 501 tarin accuracy: 0.958994708994709
Epoch 501 loss: 12.628064896911383
Epoch 476 has the best test accuracy: 0.8793650793650793

Time: 910
Evaluate epoch 502 accuracy: 0.8455026455026455
Evaluate epoch 502 precision: 0.8624078624078624
Evaluate epoch 502 recall: 0.7959183673469388
Evaluate epoch 502 F1: 0.8278301886792453
Epoch 502 tarin accuracy: 0.9574074074074074
Epoch 502 loss: 13.06784097198397
Epoch 476 has the best test accuracy: 0.8793650793650793

Time: 927
Evaluate epoch 503 accuracy: 0.837037037037037
Evaluate epoch 503 precision: 0.8046709129511678
Evaluate epoch 503 recall: 0.8594104308390023
Evaluate epoch 503 F1: 0.8311403508771931
Epoch 503 tarin accuracy: 0.9560846560846561
Epoch 503 loss: 16.513943161815405
Epoch 476 has the best test accuracy: 0.8793650793650793

Time

Time: 1332
Evaluate epoch 526 accuracy: 0.8338624338624339
Evaluate epoch 526 precision: 0.808695652173913
Evaluate epoch 526 recall: 0.8435374149659864
Evaluate epoch 526 F1: 0.8257491675915649
Epoch 526 tarin accuracy: 0.9666666666666667
Epoch 526 loss: 11.285410126671195
Epoch 476 has the best test accuracy: 0.8793650793650793

Time: 1350
Evaluate epoch 527 accuracy: 0.8275132275132275
Evaluate epoch 527 precision: 0.792016806722689
Evaluate epoch 527 recall: 0.854875283446712
Evaluate epoch 527 F1: 0.822246455834242
Epoch 527 tarin accuracy: 0.9637566137566138
Epoch 527 loss: 11.393518719822168
Epoch 476 has the best test accuracy: 0.8793650793650793

Time: 1367
Evaluate epoch 528 accuracy: 0.8518518518518519
Evaluate epoch 528 precision: 0.81419624217119
Evaluate epoch 528 recall: 0.8843537414965986
Evaluate epoch 528 F1: 0.8478260869565218
Epoch 528 tarin accuracy: 0.9534391534391534
Epoch 528 loss: 15.095608461648226
Epoch 476 has the best test accuracy: 0.8793650793650793

Time

In [ ]:
model = get_resnet()
optimizer = get_optimizer(model)
train_model(model, optimizer, start_epoch=1, epoch_num=400, filename='resnet-no-tl.txt')

Time: 26
Evaluate epoch 1 accuracy: 0.5693121693121693
Evaluate epoch 1 precision: 0.5702479338842975
Evaluate epoch 1 recall: 0.3129251700680272
Evaluate epoch 1 F1: 0.404099560761347
Epoch 1 tarin accuracy: 0.5515873015873016
Epoch 1 loss: 89.35838317871094
Epoch 1 has the best test accuracy: 0.5693121693121693

Time: 57
Evaluate epoch 2 accuracy: 0.5661375661375662
Evaluate epoch 2 precision: 0.5349887133182845
Evaluate epoch 2 recall: 0.5374149659863946
Evaluate epoch 2 F1: 0.5361990950226244
Epoch 2 tarin accuracy: 0.5603174603174603
Epoch 2 loss: 81.19277679920197
Epoch 1 has the best test accuracy: 0.5693121693121693

Time: 88
Evaluate epoch 3 accuracy: 0.564021164021164
Evaluate epoch 3 precision: 0.5466237942122186
Evaluate epoch 3 recall: 0.3854875283446712
Evaluate epoch 3 F1: 0.4521276595744681
Epoch 3 tarin accuracy: 0.567989417989418
Epoch 3 loss: 81.16956353187561
Epoch 1 has the best test accuracy: 0.5693121693121693

Time: 118
Evaluate epoch 4 accuracy: 0.5746031746031

Time: 820
Evaluate epoch 27 accuracy: 0.6074074074074074
Evaluate epoch 27 precision: 0.5841346153846154
Evaluate epoch 27 recall: 0.5510204081632653
Evaluate epoch 27 F1: 0.5670945157526254
Epoch 27 tarin accuracy: 0.6126984126984127
Epoch 27 loss: 78.55782920122147
Epoch 25 has the best test accuracy: 0.6137566137566137

Time: 851
Evaluate epoch 28 accuracy: 0.6137566137566137
Evaluate epoch 28 precision: 0.6461538461538462
Evaluate epoch 28 recall: 0.38095238095238093
Evaluate epoch 28 F1: 0.47931526390870194
Epoch 28 tarin accuracy: 0.6058201058201058
Epoch 28 loss: 78.95955663919449
Epoch 25 has the best test accuracy: 0.6137566137566137

Time: 881
Evaluate epoch 29 accuracy: 0.6105820105820106
Evaluate epoch 29 precision: 0.640926640926641
Evaluate epoch 29 recall: 0.3764172335600907
Evaluate epoch 29 F1: 0.47428571428571425
Epoch 29 tarin accuracy: 0.6005291005291006
Epoch 29 loss: 79.03300803899765
Epoch 25 has the best test accuracy: 0.6137566137566137

Time: 912
Evaluate epoc

Time: 1614
Evaluate epoch 53 accuracy: 0.6433862433862434
Evaluate epoch 53 precision: 0.6614906832298136
Evaluate epoch 53 recall: 0.48299319727891155
Evaluate epoch 53 F1: 0.5583224115334207
Epoch 53 tarin accuracy: 0.6457671957671958
Epoch 53 loss: 76.04111051559448
Epoch 51 has the best test accuracy: 0.6497354497354497

Time: 1644
Evaluate epoch 54 accuracy: 0.6412698412698413
Evaluate epoch 54 precision: 0.6482558139534884
Evaluate epoch 54 recall: 0.5056689342403629
Evaluate epoch 54 F1: 0.5681528662420383
Epoch 54 tarin accuracy: 0.6489417989417989
Epoch 54 loss: 74.99110651016235
Epoch 51 has the best test accuracy: 0.6497354497354497

Time: 1674
Evaluate epoch 55 accuracy: 0.617989417989418
Evaluate epoch 55 precision: 0.6290322580645161
Evaluate epoch 55 recall: 0.4421768707482993
Evaluate epoch 55 F1: 0.5193075898801599
Epoch 55 tarin accuracy: 0.6404761904761904
Epoch 55 loss: 75.95322436094284
Epoch 51 has the best test accuracy: 0.6497354497354497

Time: 1705
Evaluate ep

Time: 2498
Evaluate epoch 82 accuracy: 0.6507936507936508
Evaluate epoch 82 precision: 0.6707692307692308
Evaluate epoch 82 recall: 0.4943310657596372
Evaluate epoch 82 F1: 0.5691906005221933
Epoch 82 tarin accuracy: 0.6537037037037037
Epoch 82 loss: 74.11744421720505
Epoch 60 has the best test accuracy: 0.6507936507936508

Time: 2529
Evaluate epoch 83 accuracy: 0.6486772486772486
Evaluate epoch 83 precision: 0.6646525679758308
Evaluate epoch 83 recall: 0.4988662131519274
Evaluate epoch 83 F1: 0.5699481865284974
Epoch 83 tarin accuracy: 0.6523809523809524
Epoch 83 loss: 74.05577102303505
Epoch 60 has the best test accuracy: 0.6507936507936508

Time: 2559
Evaluate epoch 84 accuracy: 0.6507936507936508
Evaluate epoch 84 precision: 0.6907216494845361
Evaluate epoch 84 recall: 0.4557823129251701
Evaluate epoch 84 F1: 0.5491803278688524
Epoch 84 tarin accuracy: 0.65
Epoch 84 loss: 74.25473988056183
Epoch 60 has the best test accuracy: 0.6507936507936508

Time: 2590
Evaluate epoch 85 accurac

Time: 3292
Evaluate epoch 108 accuracy: 0.6465608465608466
Evaluate epoch 108 precision: 0.6988847583643123
Evaluate epoch 108 recall: 0.42630385487528344
Evaluate epoch 108 F1: 0.5295774647887325
Epoch 108 tarin accuracy: 0.6656084656084656
Epoch 108 loss: 72.29596558213234
Epoch 95 has the best test accuracy: 0.6592592592592592

Time: 3322
Evaluate epoch 109 accuracy: 0.6592592592592592
Evaluate epoch 109 precision: 0.6612466124661247
Evaluate epoch 109 recall: 0.5532879818594104
Evaluate epoch 109 F1: 0.6024691358024692
Epoch 109 tarin accuracy: 0.6616402116402117
Epoch 109 loss: 72.90354764461517
Epoch 95 has the best test accuracy: 0.6592592592592592

Time: 3353
Evaluate epoch 110 accuracy: 0.6582010582010582
Evaluate epoch 110 precision: 0.6446078431372549
Evaluate epoch 110 recall: 0.5963718820861678
Evaluate epoch 110 F1: 0.6195524146054181
Epoch 110 tarin accuracy: 0.6687830687830688
Epoch 110 loss: 72.16087874770164
Epoch 95 has the best test accuracy: 0.6592592592592592

Tim

...

Time: 7870
Evaluate epoch 258 accuracy: 0.7417989417989418
Evaluate epoch 258 precision: 0.7822349570200573
Evaluate epoch 258 recall: 0.6190476190476191
Evaluate epoch 258 F1: 0.6911392405063291
Epoch 258 tarin accuracy: 0.7894179894179895
Epoch 258 loss: 52.62472864985466
Epoch 242 has the best test accuracy: 0.7513227513227513

Time: 7900
Evaluate epoch 259 accuracy: 0.7333333333333333
Evaluate epoch 259 precision: 0.7723342939481268
Evaluate epoch 259 recall: 0.6077097505668935
Evaluate epoch 259 F1: 0.6802030456852792
Epoch 259 tarin accuracy: 0.791005291005291
Epoch 259 loss: 52.8738312125206
Epoch 242 has the best test accuracy: 0.7513227513227513

Time: 7931
Evaluate epoch 260 accuracy: 0.7428571428571429
Evaluate epoch 260 precision: 0.7661290322580645
Evaluate epoch 260 recall: 0.6462585034013606
Evaluate epoch 260 F1: 0.7011070110701108
Epoch 260 tarin accuracy: 0.7904761904761904
Epoch 260 loss: 52.66899345815182
Epoch 242 has the best test accuracy: 0.7513227513227513

Tim

Time: 8633
Evaluate epoch 283 accuracy: 0.7555555555555555
Evaluate epoch 283 precision: 0.7511961722488039
Evaluate epoch 283 recall: 0.7120181405895691
Evaluate epoch 283 F1: 0.731082654249127
Epoch 283 tarin accuracy: 0.81005291005291
Epoch 283 loss: 49.0546588152647
Epoch 273 has the best test accuracy: 0.7671957671957672

Time: 8664
Evaluate epoch 284 accuracy: 0.7375661375661375
Evaluate epoch 284 precision: 0.6888454011741683
Evaluate epoch 284 recall: 0.7981859410430839
Evaluate epoch 284 F1: 0.7394957983193277
Epoch 284 tarin accuracy: 0.8171957671957671
Epoch 284 loss: 48.26159608364105
Epoch 273 has the best test accuracy: 0.7671957671957672

Time: 8694
Evaluate epoch 285 accuracy: 0.7333333333333333
Evaluate epoch 285 precision: 0.6964656964656964
Evaluate epoch 285 recall: 0.7596371882086168
Evaluate epoch 285 F1: 0.7266811279826463
Epoch 285 tarin accuracy: 0.8097883597883598
Epoch 285 loss: 49.493006244301796
Epoch 273 has the best test accuracy: 0.7671957671957672

Time

Time: 9397
Evaluate epoch 308 accuracy: 0.7428571428571429
Evaluate epoch 308 precision: 0.7133620689655172
Evaluate epoch 308 recall: 0.7505668934240363
Evaluate epoch 308 F1: 0.7314917127071824
Epoch 308 tarin accuracy: 0.8124338624338624
Epoch 308 loss: 48.88102266192436
Epoch 296 has the best test accuracy: 0.7703703703703704

Time: 9427
Evaluate epoch 309 accuracy: 0.7534391534391535
Evaluate epoch 309 precision: 0.7260869565217392
Evaluate epoch 309 recall: 0.7573696145124716
Evaluate epoch 309 F1: 0.7413984461709211
Epoch 309 tarin accuracy: 0.843915343915344
Epoch 309 loss: 42.769169837236404
Epoch 296 has the best test accuracy: 0.7703703703703704

Time: 9458
Evaluate epoch 310 accuracy: 0.7216931216931217
Evaluate epoch 310 precision: 0.7908496732026143
Evaluate epoch 310 recall: 0.5487528344671202
Evaluate epoch 310 F1: 0.6479250334672022
Epoch 310 tarin accuracy: 0.8378306878306878
Epoch 310 loss: 42.74960248172283
Epoch 296 has the best test accuracy: 0.7703703703703704

T

Time: 10160
Evaluate epoch 333 accuracy: 0.7724867724867724
Evaluate epoch 333 precision: 0.7703349282296651
Evaluate epoch 333 recall: 0.7301587301587301
Evaluate epoch 333 F1: 0.7497089639115251
Epoch 333 tarin accuracy: 0.8716931216931217
Epoch 333 loss: 36.613824523985386
Epoch 332 has the best test accuracy: 0.7851851851851852

Time: 10190
Evaluate epoch 334 accuracy: 0.7841269841269841
Evaluate epoch 334 precision: 0.7724137931034483
Evaluate epoch 334 recall: 0.7619047619047619
Evaluate epoch 334 F1: 0.767123287671233
Epoch 334 tarin accuracy: 0.846031746031746
Epoch 334 loss: 41.31244823336601
Epoch 332 has the best test accuracy: 0.7851851851851852

Time: 10221
Evaluate epoch 335 accuracy: 0.7841269841269841
Evaluate epoch 335 precision: 0.7515923566878981
Evaluate epoch 335 recall: 0.8027210884353742
Evaluate epoch 335 F1: 0.7763157894736842
Epoch 335 tarin accuracy: 0.8685185185185185
Epoch 335 loss: 37.18575510382652
Epoch 332 has the best test accuracy: 0.7851851851851852


Time: 10923
Evaluate epoch 358 accuracy: 0.7724867724867724
Evaluate epoch 358 precision: 0.7156488549618321
Evaluate epoch 358 recall: 0.8503401360544217
Evaluate epoch 358 F1: 0.7772020725388601
Epoch 358 tarin accuracy: 0.8759259259259259
Epoch 358 loss: 34.999835290014744
Epoch 352 has the best test accuracy: 0.8052910052910053

Time: 10954
Evaluate epoch 359 accuracy: 0.7904761904761904
Evaluate epoch 359 precision: 0.7793103448275862
Evaluate epoch 359 recall: 0.7687074829931972
Evaluate epoch 359 F1: 0.7739726027397261
Epoch 359 tarin accuracy: 0.8933862433862434
Epoch 359 loss: 30.88558967784047
Epoch 352 has the best test accuracy: 0.8052910052910053

Time: 10984
Evaluate epoch 360 accuracy: 0.8063492063492064
Evaluate epoch 360 precision: 0.7892376681614349
Evaluate epoch 360 recall: 0.7981859410430839
Evaluate epoch 360 F1: 0.7936865839909808
Epoch 360 tarin accuracy: 0.882010582010582
Epoch 360 loss: 31.85635670274496
Epoch 360 has the best test accuracy: 0.8063492063492064

In [206]:
train_model(model, optimizer, start_epoch=601, epoch_num=50, filename='resnet-no-tl.txt')

Time: 27
Evaluate epoch 601 accuracy: 0.8952380952380953
Evaluate epoch 601 precision: 0.9110576923076923
Evaluate epoch 601 recall: 0.8594104308390023
Evaluate epoch 601 F1: 0.8844807467911319
Epoch 601 tarin accuracy: 0.9738095238095238
Epoch 601 loss: 8.41021959297359
Epoch 601 has the best test accuracy: 0.8952380952380953

Time: 57
Evaluate epoch 602 accuracy: 0.8984126984126984
Evaluate epoch 602 precision: 0.8758169934640523
Evaluate epoch 602 recall: 0.9115646258503401
Evaluate epoch 602 F1: 0.8933333333333333
Epoch 602 tarin accuracy: 0.9809523809523809
Epoch 602 loss: 6.071739820763469
Epoch 602 has the best test accuracy: 0.8984126984126984

Time: 88
Evaluate epoch 603 accuracy: 0.9047619047619048
Evaluate epoch 603 precision: 0.8874172185430463
Evaluate epoch 603 recall: 0.9115646258503401
Evaluate epoch 603 F1: 0.8993288590604027
Epoch 603 tarin accuracy: 0.9796296296296296
Epoch 603 loss: 6.806025286205113
Epoch 603 has the best test accuracy: 0.9047619047619048

Time: 11

Time: 786
Evaluate epoch 626 accuracy: 0.9142857142857143
Evaluate epoch 626 precision: 0.9147465437788018
Evaluate epoch 626 recall: 0.9002267573696145
Evaluate epoch 626 F1: 0.9074285714285715
Epoch 626 tarin accuracy: 0.9788359788359788
Epoch 626 loss: 6.460627414286137
Epoch 624 has the best test accuracy: 0.926984126984127

Time: 817
Evaluate epoch 627 accuracy: 0.9216931216931217
Evaluate epoch 627 precision: 0.9400479616306955
Evaluate epoch 627 recall: 0.8888888888888888
Evaluate epoch 627 F1: 0.9137529137529138
Epoch 627 tarin accuracy: 0.9835978835978836
Epoch 627 loss: 6.273629879578948
Epoch 624 has the best test accuracy: 0.926984126984127

Time: 848
Evaluate epoch 628 accuracy: 0.9121693121693122
Evaluate epoch 628 precision: 0.9221698113207547
Evaluate epoch 628 recall: 0.8866213151927438
Evaluate epoch 628 F1: 0.9040462427745665
Epoch 628 tarin accuracy: 0.953968253968254
Epoch 628 loss: 15.273282593116164
Epoch 624 has the best test accuracy: 0.926984126984127

Time: 8

In [ ]:
model = get_basenet()
optimizer = get_optimizer(model, lr=0.001)
train_model(model, optimizer, start_epoch=1, epoch_num=100, filename='basenet.txt')

Time: 12
Evaluate epoch 1 accuracy: 0.5481481481481482
Evaluate epoch 1 precision: 0.6842105263157895
Evaluate epoch 1 recall: 0.05895691609977324
Evaluate epoch 1 F1: 0.10855949895615866
Epoch 1 tarin accuracy: 0.5164021164021164
Epoch 1 loss: 82.4815041422844
Epoch 1 has the best test accuracy: 0.5481481481481482



Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3f870b44d0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3f870b44d0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/opt/anacond

Time: 29


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3f870b44d0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3f870b44d0>
Traceback (most recent call last):
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3f870b44d0>
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
Traceback (most recent call last):
    self._shutdown_workers()
  File "/opt/anaconda3/

Evaluate epoch 2 accuracy: 0.5576719576719577
Evaluate epoch 2 precision: 0.5268065268065268
Evaluate epoch 2 recall: 0.5124716553287982
Evaluate epoch 2 F1: 0.5195402298850577
Epoch 2 tarin accuracy: 0.5732804232804233
Epoch 2 loss: 80.98977357149124
Epoch 2 has the best test accuracy: 0.5576719576719577



...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3f870b44d0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3f870b44d0>
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3f870b44d0>
Traceback (most recent call last):
    w.join()
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
  File "/opt/anaconda3/lib/python3.7/sit

Time: 114


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3f870b44d0>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3f870b44d0>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
    self._shutdown_workers()
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
    w.join()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", 

Evaluate epoch 7 accuracy: 0.582010582010582
Evaluate epoch 7 precision: 0.5529953917050692
Evaluate epoch 7 recall: 0.54421768707483
Evaluate epoch 7 F1: 0.5485714285714286
Epoch 7 tarin accuracy: 0.5970899470899471
Epoch 7 loss: 78.9176587164402
Epoch 7 has the best test accuracy: 0.582010582010582



Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3f870b44d0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3f870b44d0>
    w.join()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3f870b44d0>
Traceback (most recent call last):
  File "/opt/anaconda3/

In [134]:
# Evaluation

def evaluate(model, data_loader, epoch):
    with torch.no_grad():
        sample_count = 0
        correct_count = 0
        
        TP = 0
        FP = 0
        FN = 0
        
        accuracy = 0.0
        precision = 0.0
        recall = 0.0
        F1 = 0.0

        for i, samples in enumerate(data_loader):
            images, labels = samples

            if IS_GPU:
                images = images.cuda()
                labels = labels.cuda()

            outputs = model(images)
            _, preds = torch.max(outputs.data, 1)

            sample_count += len(labels)
            correct_count += torch.sum(preds == labels).item()
            
            for idx in range(len(labels)):
                if labels[idx] == 1 and preds[idx] == 1:
                    TP += 1
                if labels[idx] == 1 and preds[idx] == 0:
                    FN += 1
                if labels[idx] == 0 and preds[idx] == 1:
                    FP += 1
        
        try:
            accuracy = correct_count / sample_count
            precision = TP / (TP + FP)
            recall = TP / (TP + FN)
            F1 = 2 * precision * recall / (precision + recall)

            print('Evaluate epoch {} accuracy: '.format(epoch) + str(accuracy))
            print('Evaluate epoch {} precision: '.format(epoch) + str(precision))
            print('Evaluate epoch {} recall: '.format(epoch) + str(recall))
            print('Evaluate epoch {} F1: '.format(epoch) + str(F1))
        except ZeroDivisionError:
            pass
        
        return accuracy, precision, recall, F1
        
evaluate(model, test_loader, 0)

Evaluate epoch 0 accuracy: 0.5322751322751322
Evaluate epoch 0 precision: 0.4972972972972973
Evaluate epoch 0 recall: 0.20861678004535147
Evaluate epoch 0 F1: 0.29392971246006394


(0.5322751322751322,
 0.4972972972972973,
 0.20861678004535147,
 0.29392971246006394)